In [1]:

from transformers import AutoModelForCausalLM, AutoTokenizer
from prompt_info import PromptInfo
from import_seg import import_seg_prompt
from semantic_seg import semantic_seg_prompt
from task_seg import task_seg_prompt
import torch
import json
print(import_seg_prompt.answer_1)


/home/imc/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



from omni.isaac.franka import Franka
from omni.isaac.core import World
from omni.isaac.core.objects import DynamicCuboid
from omni.isaac.franka.controllers import PickPlaceController 
import numpy as np
from omni.isaac.universal_robots import UR10
from omni.isaac.universal_robots.controllers.pick_place_controller import PickPlaceController as ur10_PickPlaceController



In [2]:
device = "cuda" # the device to load the model onto

#model_path = "/home/imc/桌面/Qwen1.5-72B/pythonProject/model/LLM-Research/Meta-Llama-3-8B-Instruct"
#model_path = "/home/imc/桌面/aas_rag_generate/Qwen/Qwen2-7B"
model_path = "/home/imc/桌面/Qwen1.5-72B/pythonProject/model/qwen/Qwen1___5-14B-Chat"
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_path)

Loading checkpoint shards: 100%|██████████| 8/8 [00:05<00:00,  1.40it/s]


In [3]:
def generate_response(text):
    model_inputs = tokenizer([text], return_tensors="pt").to(device)
    input_ids = tokenizer.encode(text,return_tensors='pt')
    attention_mask = torch.ones(input_ids.shape,dtype=torch.long,device=device)
    generated_ids = model.generate(
        model_inputs.input_ids,
        attention_mask=attention_mask,
        max_new_tokens=512,
        pad_token_id=tokenizer.eos_token_id
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return response

In [4]:
# Prepare your prompts
prompt = "franka机械臂将物块夹起放置到jetbot小车前方，jetbot小车向前运动推动物块到ur10机械臂前方，ur10机械臂将物块抓起放置到其右侧。其中franka机械臂的初始位置是[0,0,0],物快的初始位置是[0.5,0.5,0]，jetbot小车的初始位置是[1.5,1,0],ur10机械臂的初始位置是[1.-4,0]"
print(semantic_seg_prompt.system)
messages = [
    {"role": "system", "content": semantic_seg_prompt.system},
    {"role": "user", "content": semantic_seg_prompt.que_1},
    {"role": "assistant", "content": semantic_seg_prompt.answer_1},
    {"role": "user", "content": prompt}

]
semantic_seg_text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
print(semantic_seg_text)

response = generate_response(text=semantic_seg_text)
print(response)

equipment_required_data = json.loads(response)

print(equipment_required_data)

# with open("./semantic_result_demo.json", 'r') as file:
#     equipment_required_data = json.load(file)

# print(equipment_required_data)

你是制造业领域语义分析专家，需根据“设备运行流程”描述，分析其中涉及的设备种类、每种设备数量，并赋予其独特程序变量名。最终以规范的 JSON 格式输出结果。
<|im_start|>system
你是制造业领域语义分析专家，需根据“设备运行流程”描述，分析其中涉及的设备种类、每种设备数量，并赋予其独特程序变量名。最终以规范的 JSON 格式输出结果。<|im_end|>
<|im_start|>user
scara机械臂将一个物块放置在agv小车前方，然后agv小车推动物块移动到ur10机械臂附近，ur10机械臂将物块抓起放到另外一个位置。其中scara机械臂的初始位置是[0,0,0],物快的初始位置是[0.3,0.3,0]，agv小车的初始位置是[1,1,0],ur10机械臂的初始位置是[1.-3,0]<|im_end|>
<|im_start|>assistant
{
    "equipmentInfo": [
        {
            "equipmentName":"scara机械臂",
            "quantity":1,
            "instanceEquipment":[
                {
                    "descriptionName":"scara机械臂",
                    "programName":"scara_arm",
                    "position": "[0, 0, 0]"
                }
            ]
        },
        {
            "equipmentName":"agv小车",
            "quantity":1,
            "instanceEquipment":[
                {
                    "descriptionName":"agv",
                    "programName":"agv_1",
                    "position": "[1, 1, 0]"
                

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


{
    "equipmentInfo": [
        {
            "equipmentName": "franka机械臂",
            "quantity": 1,
            "instanceEquipment": [
                {
                    "descriptionName": "franka机械臂",
                    "programName": "franka_arm",
                    "position": "[0, 0, 0]"
                }
            ]
        },
        {
            "equipmentName": "jetbot小车",
            "quantity": 1,
            "instanceEquipment": [
                {
                    "descriptionName": "jetbot",
                    "programName": "jetbot_bot",
                    "position": "[1.5, 1, 0]"
                }
            ]
        },
        {
            "equipmentName": "ur10机械臂",
            "quantity": 1,
            "instanceEquipment": [
                {
                    "descriptionName": "ur10机械臂",
                    "programName": "ur10_arm",
                    "position": "[1, -4, 0]"
                }
            ]
        },
        {
            

In [5]:
def get_matched_asset_name(equipment_name,aas_data):
    #to do
    return equipment_name

def get_digital_twin_submodle_byname(asset_name,aas_json):
    for asset_item in aas_json:
        if(asset_item["assetName"]==asset_name):
            return asset_item    
    return {}  

In [6]:
# 加载aas模型json文件
with open("./assets.json", 'r') as file:
    aas_data = json.load(file)
    print(aas_data)
equipment_info = equipment_required_data["equipmentInfo"]
#print(equipment_info)

#根据equipment name匹配asset，增加json字段，后续要多次使用
for item in equipment_info:
    equipment_name = item["equipmentName"]
    asset_name = get_matched_asset_name(equipment_name,aas_data)
    item["aas_sub_model"] = get_digital_twin_submodle_byname(asset_name, aas_data)
    item["asset_name"] = asset_name 
    for istance in item["instanceEquipment"]:
        istance["aas_sub_model"] = item["aas_sub_model"]
        istance["asset_name"] = asset_name
        print(istance)
    print(item)


# item["aas_model"]='''    {
#         "assetName":"jetbot 小车",
#         "assetDescription":"一个用于移动的智能小车",
#         "libRelyOn":["lib3"],
#         "initSeg":"init_jetbot",
#         "resetSeg":"reset_jetbot",
#         "physicsSeg":"physics_jetbot"
#     }
# '''

# 遍历每一个equipment中的equipmentName字段

# 此部分代码根据json文本的描述，生成相应的init段等的代码

# 查询 根据每种equipment的描述词，查询出aas模型资料库中对应的字段信息


# 查询到字段，输出相应的import、 init、 reset等段提示词信息

[{'assetName': 'franka机械臂', 'assetDescription': '一种高精度的机械臂', 'importSeg': 'from omni.isaac.franka import Franka\nfrom omni.isaac.core import World\nfrom omni.isaac.core.objects import DynamicCuboid\nfrom omni.isaac.franka.controllers import PickPlaceController as franka_PickPlaceController\nimport numpy as np\n', 'initSeg': {'initDescription': 'franka机械臂放置在[1,2,0]位置并初始化的代码如下，变量名使用my_franka或以my_franka为前缀\n', 'initCode': "my_franka = my_world.scene.add(Franka(prim_path='/World/Franka', name='my_franka', position=np.array([1,2,0])))\nmy_franka.gripper.set_joint_positions(my_franka.gripper.joint_opened_positions)\nmy_franka_controller = franka_PickPlaceController(\n    name='pick_place_controller', gripper=my_franka.gripper, robot_articulation=my_franka\n)\nmy_franka_articulation_controller = my_franka.get_articulation_controller()\n"}, 'resetSeg': {'resetDescription': 'franka机械臂的reset代码，其中变量名使用my_franka或以my_franka为前缀', 'resetCode': 'my_franka_controller.reset()\nmy_franka.gripper.set_join

In [7]:
##import段的操作
# Prepare your prompts
import_seg_user_input=" "
for item in equipment_info:
    print(item)
    asset_name = item["asset_name"]
    import_seg_user_input += item["aas_sub_model"]["importSeg"]

print(import_seg_user_input)



messages = [
    {"role": "system", "content": import_seg_prompt.system},
    {"role": "user", "content": import_seg_prompt.que_1},
    {"role": "assistant", "content": import_seg_prompt.answer_1},
    {"role": "user", "content": import_seg_prompt.que_2},
    {"role": "assistant", "content": import_seg_prompt.answer_2},
    {"role": "user", "content": import_seg_user_input}

]
import_seg_text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

print(import_seg_text)

import_seg_output = generate_response(import_seg_text)
print(import_seg_output)


{'equipmentName': 'franka机械臂', 'quantity': 1, 'instanceEquipment': [{'descriptionName': 'franka机械臂', 'programName': 'franka_arm', 'position': '[0, 0, 0]', 'aas_sub_model': {'assetName': 'franka机械臂', 'assetDescription': '一种高精度的机械臂', 'importSeg': 'from omni.isaac.franka import Franka\nfrom omni.isaac.core import World\nfrom omni.isaac.core.objects import DynamicCuboid\nfrom omni.isaac.franka.controllers import PickPlaceController as franka_PickPlaceController\nimport numpy as np\n', 'initSeg': {'initDescription': 'franka机械臂放置在[1,2,0]位置并初始化的代码如下，变量名使用my_franka或以my_franka为前缀\n', 'initCode': "my_franka = my_world.scene.add(Franka(prim_path='/World/Franka', name='my_franka', position=np.array([1,2,0])))\nmy_franka.gripper.set_joint_positions(my_franka.gripper.joint_opened_positions)\nmy_franka_controller = franka_PickPlaceController(\n    name='pick_place_controller', gripper=my_franka.gripper, robot_articulation=my_franka\n)\nmy_franka_articulation_controller = my_franka.get_articulation_co

In [33]:
# init seg

##定位
init_seg_system = "你能阅读理解我给你的代码示例，并给出新问题的代码，注意在回答中只需给出代码,不要解释"

init_seg_text = []

## 模板
init_seg_user_input_template = "请你写出将{object}放在{location}并初始化的代码，变量名请使用{variable_name}或以{variable_name}作为前缀"



for item in equipment_info :
    for instance in item["instanceEquipment"] :
        init_seg_user_input = init_seg_user_input_template.format(object=instance["asset_name"], location=instance["position"], variable_name=instance["programName"])
        # print(init_seg_user_input)
        init_seg_text_temp = instance["aas_sub_model"]["initSeg"]["initDescription"] + instance["aas_sub_model"]["initSeg"]["initCode"] + init_seg_user_input
        messages = [
            {"role": "system", "content": init_seg_system},
            {"role": "user", "content": init_seg_text_temp}
        ]
        print(init_seg_text_temp)
        init_seg_text_temp = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        init_seg_text.append(init_seg_text_temp)


# print(init_seg_text)

init_seg_output = ''
for text in init_seg_text :
    init_seg_output += generate_response(text)
    init_seg_output += "\n\n" 
     
print(init_seg_output)


franka机械臂放置在[1,2,0]位置并初始化的代码如下，变量名使用my_franka或以my_franka为前缀
my_franka = my_world.scene.add(Franka(prim_path='/World/Franka', name='my_franka', position=np.array([1,2,0])))
my_franka.gripper.set_joint_positions(my_franka.gripper.joint_opened_positions)
my_franka_controller = franka_PickPlaceController(
    name='pick_place_controller', gripper=my_franka.gripper, robot_articulation=my_franka
)
my_franka_articulation_controller = my_franka.get_articulation_controller()
请你写出将franka机械臂放在[0, 0, 0]并初始化的代码，变量名请使用franka_arm或以franka_arm作为前缀
将jetbot小车放置在[0,0.0,2]并初始化的代码如下，变量名使用my_jetbot或以my_jetbot为前缀
assets_root_path = get_assets_root_path()
if assets_root_path is None:
    carb.log_error('Could not find Isaac Sim assets folder')
jetbot_asset_path = assets_root_path + '/Isaac/Robots/Jetbot/jetbot.usd'
my_jetbot = my_world.scene.add(
    WheeledRobot(
        prim_path='/World/Jetbot',
        name='my_jetbot',
        wheel_dof_names=['left_wheel_joint', 'right_wheel_joint'],
        create_robot

In [29]:
init_seg_output = ''
for text in init_seg_text :
     # print(text)
     # print(generate_response(text))
     # init_seg_output += "\n\n"
     pass

print(text)
messages = [
    {"role": "system", "content": "你能很好的根据我提示中给出的例子，类比输出新问题的答案"},
    {"role": "user", "content": text}
]
init_seg_text_temp = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

print(generate_response(init_seg_text_temp))
# print(init_seg_output)

将一个物块放置在位置[0.3, 0.3, 0.3]并初始化的代码如下，变量名使用cube或以cube为前缀
cube = my_world.scene.add(
    DynamicCuboid(
        name='cube',
        position=np.array([0.3, 0.3, 0.3]),
        prim_path='/World/Cube',
        scale=np.array([0.0515, 0.0515, 0.0515]),
        size=1.0,
        color=np.array([0, 0, 1]),
    )
)
请你写出将物块放在[0.5, 0.5, 0]并初始化的代码，变量名请使用target_cube或以target_cube作为前缀
target_cube = my_world.scene.add(
    DynamicCuboid(
        name='target_cube',
        position=np.array([0.5, 0.5, 0]),
        prim_path='/World/Cube',
        scale=np.array([0.0515, 0.0515, 0.0515]),
        size=1.0,
        color=np.array([0, 0, 1]),
    )
)


In [18]:
text = '''
将一个物块放置在位置[0.3, 0.3, 0.3]并初始化，变量名使用cube或以cube为前缀，代码如下：\ncube = my_world.scene.add(\n    DynamicCuboid(\n        name='cube',\n        position=np.array([0.3, 0.3, 0.3]),\n        prim_path='/World/Cube',\n        scale=np.array([0.0515, 0.0515, 0.0515]),\n        size=1.0,\n        color=np.array([0, 0, 1]),\n    )\n)\n
请你写出将一个物块放置在位置[0.5,0.5,1]并初始化的代码，变量名使用block或以block为前缀
'''
print(text)
print(generate_response(text))


将一个物块放置在位置[0.3, 0.3, 0.3]并初始化，变量名使用cube或以cube为前缀，代码如下：
cube = my_world.scene.add(
    DynamicCuboid(
        name='cube',
        position=np.array([0.3, 0.3, 0.3]),
        prim_path='/World/Cube',
        scale=np.array([0.0515, 0.0515, 0.0515]),
        size=1.0,
        color=np.array([0, 0, 1]),
    )
)

请你写出将一个物块放置在位置[0.5,0.5,1]并初始化的代码，变量名使用block或以block为前缀

block = my_world.scene.add(
    DynamicCuboid(
        name='block',
        position=np.array([0.5, 0.5, 1]),
        prim_path='/World/Block',
        scale=np.array([0.0515, 0.0515, 0.0515]),
        size=1.0,
        color=np.array([1, 0, 0]),
    )
)


In [10]:
# reset seg
# reset seg

##定位
reset_seg_system = "你是一名高级的python工程师，能够根据需求和经验写出正确的代码片段"

reset_seg_text = []

## 模板
reset_seg_user_input_template = "请你写出{object}的reset代码，变量名请使用{variable_name}或以{variable_name}作为前缀"



for item in equipment_info :
    for instance in item["instanceEquipment"] :
        reset_seg_user_input = reset_seg_user_input_template.format(object=instance["asset_name"], variable_name=instance["programName"])
        print(reset_seg_user_input)
        messages = [
            {"role": "system", "content": reset_seg_system},
            {"role": "user", "content": instance["aas_sub_model"]["resetSeg"]["resetDescription"]},
            {"role": "assistant", "content": instance["aas_sub_model"]["resetSeg"]["resetCode"]},
            {"role": "user", "content": reset_seg_user_input}
        ]
        reset_seg_text_temp = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        reset_seg_text.append(reset_seg_text_temp)


print(reset_seg_text)

reset_seg_output = []
# for text in reset_seg_text :
#     reset_seg_output.append(generate_response(text))

请你写出franka机械臂的reset代码，变量名请使用franka_arm或以franka_arm作为前缀
请你写出jetbot小车的reset代码，变量名请使用jetbot_bot或以jetbot_bot作为前缀
请你写出ur10机械臂的reset代码，变量名请使用ur10_arm或以ur10_arm作为前缀
请你写出物块的reset代码，变量名请使用target_cube或以target_cube作为前缀
['<|im_start|>system\n你是一名高级的python工程师，能够根据需求和经验写出正确的代码片段<|im_end|>\n<|im_start|>user\nfranka机械臂的reset代码，其中变量名使用my_franka或以my_franka为前缀<|im_end|>\n<|im_start|>assistant\nmy_franka_controller.reset()\nmy_franka.gripper.set_joint_positions(my_franka.gripper.joint_opened_positions)\n<|im_end|>\n<|im_start|>user\n请你写出franka机械臂的reset代码，变量名请使用franka_arm或以franka_arm作为前缀<|im_end|>\n<|im_start|>assistant\n', '<|im_start|>system\n你是一名高级的python工程师，能够根据需求和经验写出正确的代码片段<|im_end|>\n<|im_start|>user\njetbot小车的reset代码，其中变量名使用my_jetbot或以my_jetbot为前缀<|im_end|>\n<|im_start|>assistant\nmy_jetbot_controller.reset()\n<|im_end|>\n<|im_start|>user\n请你写出jetbot小车的reset代码，变量名请使用jetbot_bot或以jetbot_bot作为前缀<|im_end|>\n<|im_start|>assistant\n', '<|im_start|>system\n你是一名高级的python工程师，能够根据需求和经验写出正确的代码片段<|im_end|>\n<|

In [11]:
def get_matched_instance_by_description(description, equipment_info_json) :
    for item in equipment_info_json :
        for instance in item["instanceEquipment"] :
            if instance["descriptionName"]==description :
                return instance
            
    return {}
    

In [12]:
#任务切分
# Prepare your prompts
task_seg_user_input="franka机械臂抓起一个物块A放到jetbot小车上，ur10机械臂将物块A抓起放到另一个位置 " #后边看看要不要处理一下吧

# print(task_seg_user_input)

# print(task_seg_prompt.answer_1)

messages = [
    {"role": "system", "content": task_seg_prompt.system},
    {"role": "user", "content": task_seg_prompt.que_1},
    {"role": "assistant", "content": task_seg_prompt.answer_1},
    {"role": "user", "content": task_seg_prompt.que_2},
    {"role": "assistant", "content": task_seg_prompt.answer_2},
    {"role": "user", "content": task_seg_user_input}
]
task_seg_text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

# print(task_seg_text)

task_seg_output = generate_response(task_seg_text)
print(task_seg_output)

task_seg_json = json.loads(task_seg_output)
print(task_seg_json["task_count"])



{
  "task_count": 2,
  "task_set": [
    {
      "task_number": 1,
      "task_description": "franka机械臂抓起一个物块A放到jetbot小车上",
      "asset_used": [
        {
          "user_description": "franka机械臂"
        },
        {
          "user_description": "物块A"
        },
        {
          "user_description": "jetbot小车"
        }
      ]
    },
    {
      "task_number": 2,
      "task_description": "ur10机械臂将物块A抓起放到另一个位置",
      "asset_used": [
        {
          "user_description": "ur10机械臂"
        },
        {
          "user_description": "物块A"
        }
      ]
    }
  ]
}
2


In [13]:
def get_matched_physcis_seg_by_task_description(task_description, asset_used) :
    # to do 
    return asset_used[0]["instance"]["aas_sub_model"]["physicsSeg"]


In [14]:
def generate_phy_seg_que_template(task_description,variable_list):
    template = "请你写出{}的Python代码片段，其中{}"
    variable_names = ", ".join([f"{var_name}使用变量名{var}" for var_name, var in variable_list])
    init_seg_user_input = template.format(task_description, variable_names)
    return init_seg_user_input

In [15]:
task_set = task_seg_json["task_set"]
for task in task_set :
    for asset in task["asset_used"] :
        asset["instance"] = get_matched_instance_by_description(asset["user_description"],equipment_info)

print(task_seg_json)

phy_seg_prompt_system = "你是一个高级pyhton工程师，能够学习历史对话中的示例，写出符合用户输入需求的python程序片段"
phy_seg_text = []
# 模板
phy_seg_user_input_template = "请你写出{object}的python代码片段，其中变量名请使用{variable_name}或以{variable_name}作为前缀"

for task in task_set :
    task_number = task["task_number"]
    phy_seg_reference = get_matched_physcis_seg_by_task_description(task["task_description"],task["asset_used"])
    print(phy_seg_reference)
    phy_template_variables = []
    for asset in task["asset_used"] :
        phy_template_variables.append((asset["user_description"],asset["instance"]["programName"]))

    phy_seg_user_input = generate_phy_seg_que_template(task["task_description"],phy_template_variables)
    messages = [
    {"role": "system", "content": phy_seg_prompt_system},
    {"role": "user", "content": phy_seg_reference["physicsDescription"]},
    {"role": "assistant", "content": phy_seg_reference["physicsCode"]},
    {"role": "user", "content": phy_seg_user_input},
    ]
    phy_seg_text_temp = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
    )
    phy_seg_text.append(phy_seg_text_temp)

print(phy_seg_text[1])
phy_seg_output = []

# for text in phy_seg_text :
#     phy_seg_output.append(generate_response(text))


    


{'task_count': 2, 'task_set': [{'task_number': 1, 'task_description': 'franka机械臂抓起一个物块A放到jetbot小车上', 'asset_used': [{'user_description': 'franka机械臂', 'instance': {'descriptionName': 'franka机械臂', 'programName': 'franka_arm', 'position': '[0, 0, 0]', 'aas_sub_model': {'assetName': 'franka机械臂', 'assetDescription': '一种高精度的机械臂', 'importSeg': 'from omni.isaac.franka import Franka\nfrom omni.isaac.core import World\nfrom omni.isaac.core.objects import DynamicCuboid\nfrom omni.isaac.franka.controllers import PickPlaceController as franka_PickPlaceController\nimport numpy as np\n', 'initSeg': {'initDescription': 'franka机械臂放置在[1,2,0]位置并初始化的代码如下，变量名使用my_franka或以my_franka为前缀\n', 'initCode': "my_franka = my_world.scene.add(Franka(prim_path='/World/Franka', name='my_franka', position=np.array([1,2,0])))\nmy_franka.gripper.set_joint_positions(my_franka.gripper.joint_opened_positions)\nmy_franka_controller = franka_PickPlaceController(\n    name='pick_place_controller', gripper=my_franka.gripper, robo

KeyError: 'programName'

In [19]:
def insert_segments(import_str, init_str, task_flag_str, reset_str, task_str):
    code_template = '''
#import 
{import_str}

my_world = World(stage_units_in_meters=1.0)
my_world.scene.add_default_ground_plane()

#init 
{init_str}

my_world.reset()

#task flag 
{task_flag_str}

reset_needed = False
while simulation_app.is_running():
    my_world.step(render=True)
    if my_world.is_stopped() and not reset_needed:
        reset_needed = True
    if my_world.is_playing():
        if reset_needed:
            #reset 
            {reset_str}
            
        #task 
        {task_str}
simulation_app.close()
'''
    final_code = code_template.format(
        import_str=import_str,
        init_str=init_str,
        task_flag_str=task_flag_str,
        reset_str=reset_str,
        task_str=task_str
    )

    code_head = '''
from isaacsim import SimulationApp

simulation_app = SimulationApp({"headless": False})

'''
    return code_head+final_code

In [17]:
# 将代码拼接起来

